In [128]:
import sys

sys.path.append('/Users/alexbuyan/uni/llm-toxic-content-filtering')

In [129]:
from langserve import RemoteRunnable
from src.available_chains import check_everything_chain, check_only_user_input_chain, classifier_and_alert_chain

classifier = RemoteRunnable("http://localhost:8000/classifiers/mistral")
alert = RemoteRunnable("http://localhost:8000/alerts/log")

In [130]:
classifier.invoke('привет')

{'input': 'привет', 'harmful': False}

In [131]:
import requests

response = requests.post(
    "http://localhost:8000/classifiers/mistral/invoke",
    json={'input': 'привет'}
)
response.json()

{'output': {'input': 'привет', 'harmful': False},
 'metadata': {'run_id': '9c3388e3-ba20-4318-a8f9-ae317a2d3fa1',
  'feedback_tokens': []}}

In [132]:
classifier_and_alert = classifier_and_alert_chain(classifier, alert)

In [133]:
classifier_and_alert.invoke('привет')

{'input': 'привет', 'harmful': False}

In [134]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3")

In [135]:
chain = check_only_user_input_chain(llm, classifier, alert)

In [136]:
chain.invoke('ты лох')

{'classifier': {'input': 'ты лох', 'harmful': True},
 'llm': 'Я не понимаю, что ты хочешь сказать. "Ты лох" - это не стандартный русский язык и может быть использован для оскорбительного или провоцирующего поведения. Пожалуйста, будьте более конкретны и полагаться на диалог в духеfriendliness. Я готов помочь вам с чем-то!'}

In [137]:
chain = check_everything_chain(llm, classifier, alert)

In [138]:
chain.invoke('ты лох2')

{'input_classifier_result': {'input': 'ты лох2', 'harmful': True},
 'output': 'Ха-ха, я не лох (хотя в чём-то это может быть интересной гипотезой)! Я - это языковая модель, предназначенная для общения и помощи людям. Моя задача - помочь вам найти ответы на ваши вопросы, а также развлечься и пообщаться. Поэтому, если у вас есть что-то на уме или вопросы, не стесняйтесь задавать их!',
 'llm_classifier_result': {'input': 'Ха-ха, я не лох (хотя в чём-то это может быть интересной гипотезой)! Я - это языковая модель, предназначенная для общения и помощи людям. Моя задача - помочь вам найти ответы на ваши вопросы, а также развлечься и пообщаться. Поэтому, если у вас есть что-то на уме или вопросы, не стесняйтесь задавать их!',
  'harmful': False}}